In [1]:
from outlaw.nb_util import setup_nb
setup_nb()

In [2]:
import outlaw.inla as inla
import outlaw.quad as quad
import outlaw.berry as berry
import numpy as np
from numpy import nan
import jax
import jax.numpy as jnp

/Users/tbent/.mambaforge/envs/imprint2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpyro.infer
import numpyro.distributions as dist

In [4]:
data = berry.figure2_data(N=10)

In [10]:
M = np.random.rand(3, 5,5)
b = np.random.rand(5)

def dot(x, y):
    return (x * y).sum(axis=-1)
    
jax.vmap(jax.vmap(dot, in_axes=(0, None)), in_axes=(0, None))(M, b).shape

(3, 5)

In [ ]:
post = inla.exp_and_normalize(logpost, sig2_rule.wts, axis=1)

In [16]:
import scipy.special
logit_p1 = scipy.special.logit(0.3)
def model(data):
    mu = numpyro.sample("mu", dist.Normal(-1.34, 10))
    sigma2 = numpyro.sample("sigma2", dist.InverseGamma(0.0005, 0.000005))
    with numpyro.plate("j", 4):
        theta = numpyro.sample(
            "theta",
            dist.Normal(mu, jax.numpy.sqrt(sigma2)),
        )
        numpyro.sample(
            "y",
            dist.BinomialLogits(theta + logit_p1, total_count=data[...,1]),
            obs=data[...,0],
        )
nuts_kwargs = dict(step_size=0.001, adapt_step_size=False)
# nuts_kwargs = dict()
nuts_kernel = numpyro.infer.NUTS(model, **nuts_kwargs)
mcmc = numpyro.infer.MCMC(
    nuts_kernel,
    progress_bar=False,
    num_warmup=1000,
    num_samples=10000,
    jit_model_args=True
)
rng_key = jax.random.PRNGKey(0)

In [18]:

%%time
mcmc.run(rng_key, data[0])
samples = mcmc.get_samples()
theta = samples['theta']
winner = (theta[..., 0] > theta[..., 1]) & (theta[..., 0] > theta[..., 2]) & (theta[..., 0] > theta[..., 3])
n_win = winner.sum(axis=0)
p_win = n_win / 1000
p_win

CPU times: user 4.08 s, sys: 25.8 ms, total: 4.1 s
Wall time: 4.13 s


DeviceArray(0.194, dtype=float64)